In [9]:
import sys
sys.path.insert(0, '..')

In [10]:
import torch
from src.models import resnet_12
import re
from src.optimizers import modified_sgd
import os

In [13]:
# chkpt = torch.load('./runs/metal_tiered_r12_SVM_n5s15q6tb8_SGD0.1Drop204050/chkpt_055.pt',
#                    map_location=torch.device("cuda"))
chkpt = torch.load('../runs/tiered_PN_try/chkpt_055.pt',
                   map_location=torch.device("cuda"))

In [3]:
chkpt.keys()

dict_keys(['model', 'optimizer'])

In [4]:
model = resnet_12.resnet12(avg_pool=False, drop_rate=0.1, dropblock_size=2,
                num_classes=0, classifier_type='no-classifier',
                projection=False, learnable_scale=True)

Unit norm projection is  False
Average pooling:  False


In [5]:
optimizer = modified_sgd.SGD([
    {'params': model.parameters(), 'lr': 0.1,
    'weight_decay': 0.0005, 'momentum': 0.9, 'nesterov': True},
])

In [6]:
model_dict = model.state_dict() # new model's state dict
chkpt_state_dict = chkpt['model']
chkpt_state_dict_old_keys = list(chkpt_state_dict.keys())
for key in chkpt_state_dict_old_keys:
    if 'module.' in key:
        new_key = re.sub('module\.', '',  key)
        chkpt_state_dict[new_key] = chkpt_state_dict.pop(key)
load_model_state_dict = {k: v for k, v in chkpt_state_dict.items() if k in model_dict}
model_dict.update(load_model_state_dict)

print(f"Updated {len(load_model_state_dict.keys())} keys using chkpt")
print("Following keys updated:")
print("\n".join(sorted(load_model_state_dict.keys())))
missed_keys = set(model_dict).difference(set(load_model_state_dict))

print()
print(f"Missed {len(missed_keys)} keys")
print("Following keys missed:", "\n".join(sorted(missed_keys)))
model.load_state_dict(model_dict)

Updated 100 keys using chkpt
Following keys updated:
layer1.0.bn1.bias
layer1.0.bn1.num_batches_tracked
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.weight
layer1.0.bn2.bias
layer1.0.bn2.num_batches_tracked
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.weight
layer1.0.bn3.bias
layer1.0.bn3.num_batches_tracked
layer1.0.bn3.running_mean
layer1.0.bn3.running_var
layer1.0.bn3.weight
layer1.0.conv1.weight
layer1.0.conv2.weight
layer1.0.conv3.weight
layer1.0.downsample.0.weight
layer1.0.downsample.1.bias
layer1.0.downsample.1.num_batches_tracked
layer1.0.downsample.1.running_mean
layer1.0.downsample.1.running_var
layer1.0.downsample.1.weight
layer1.0.num_batches_tracked
layer2.0.bn1.bias
layer2.0.bn1.num_batches_tracked
layer2.0.bn1.running_mean
layer2.0.bn1.running_var
layer2.0.bn1.weight
layer2.0.bn2.bias
layer2.0.bn2.num_batches_tracked
layer2.0.bn2.running_mean
layer2.0.bn2.running_var
layer2.0.bn2.weight
layer2.0.bn3.bias
layer2.0.bn3.num_batches_tra

<All keys matched successfully>

In [12]:
chkpt['model']['layer1.0.conv1.weight']

tensor([[[[-1.7346e-03, -2.0156e-03, -2.3180e-03],
          [-1.7368e-03, -1.7394e-03, -1.7171e-03],
          [-1.8053e-03, -1.5829e-03, -1.2368e-03]],

         [[ 1.9253e-03,  6.6621e-04,  8.0381e-04],
          [ 1.3739e-03,  3.6057e-05,  6.8232e-04],
          [ 1.2967e-03,  6.4752e-04,  1.1699e-03]],

         [[ 4.7446e-03,  3.0351e-03,  3.2015e-03],
          [ 3.9372e-03,  2.0797e-03,  2.7595e-03],
          [ 3.6650e-03,  2.4927e-03,  2.9630e-03]]],


        [[[ 6.5395e-04,  6.2875e-03,  9.1429e-03],
          [-9.6274e-04,  3.6382e-02,  2.9371e-02],
          [ 1.6544e-02, -7.4395e-02, -1.6821e-02]],

         [[-4.7016e-03,  6.1646e-03,  7.6912e-03],
          [ 1.9596e-03,  5.7076e-02,  3.2797e-02],
          [ 3.5127e-02, -9.8689e-02, -2.8160e-02]],

         [[-7.3366e-03,  1.7887e-02,  1.6063e-02],
          [ 4.8400e-03,  3.1805e-02,  2.0513e-02],
          [ 1.3667e-02, -5.4773e-02, -3.2032e-02]]],


        [[[ 8.1392e-03, -2.6531e-03, -2.2012e-03],
          [ 6.0

In [19]:
model.module.layer1[0].conv1.weight.get_device()

0

In [16]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1'
print('Using GPUs: ', os.environ["CUDA_VISIBLE_DEVICES"])
model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()

Using GPUs:  0,1


DataParallel(
  (module): ResNet(
    (layer1): Sequential(
      (0): BasicBlock(
        (relu): LeakyReLU(negative_slope=0.1)
        (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (downsample): Sequential(
          (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [40]:
optimizer.param_groups[0]['params'][0].is_cuda

False

In [7]:
print(optimizer.param_groups[0]['params'][0].__hash__())
print(id(optimizer.param_groups[0]['params'][0])) # torch.nn.parameter.Parameter are a subclass of torch.Tensor, which returns id() for hash values
# so two different torch.Tensors despite having the same content will not hash to the same location if they are in different addresses.

140600999949648
140600999949648


In [8]:
optimizer.state = chkpt['optimizer'].state

In [20]:
print(optimizer.param_groups[0]['params'][0] in optimizer.state)

False


In [39]:
optimizer.load_state_dict(chkpt['optimizer'].state_dict())

In [28]:
print(optimizer.param_groups[0]['params'][0] in optimizer.state)

True


In [30]:
for key in optimizer.state.keys():
    print(optimizer.state[key]['exp_avg'].get_device())
    break

-1


In [35]:
type(optimizer.state[key]['exp_avg'])

torch.Tensor

In [12]:
# for key in optimizer.state.keys():
#     if key == optimizer.param_groups[0]['params'][0]:
#         print(key)

In [13]:
for key in optimizer.state:
    print(key.__hash__())
    print(type(key))

140600999949648
<class 'torch.nn.parameter.Parameter'>
140600999842592
<class 'torch.nn.parameter.Parameter'>
140600999842752
<class 'torch.nn.parameter.Parameter'>
140600999842352
<class 'torch.nn.parameter.Parameter'>
140600999842912
<class 'torch.nn.parameter.Parameter'>
140600999844992
<class 'torch.nn.parameter.Parameter'>
140600999870368
<class 'torch.nn.parameter.Parameter'>
140600999869568
<class 'torch.nn.parameter.Parameter'>
140600999869168
<class 'torch.nn.parameter.Parameter'>
140600999869008
<class 'torch.nn.parameter.Parameter'>
140600999843792
<class 'torch.nn.parameter.Parameter'>
140600999844352
<class 'torch.nn.parameter.Parameter'>
140600999843552
<class 'torch.nn.parameter.Parameter'>
140600999890688
<class 'torch.nn.parameter.Parameter'>
140600999890048
<class 'torch.nn.parameter.Parameter'>
140600999889968
<class 'torch.nn.parameter.Parameter'>
140600999828208
<class 'torch.nn.parameter.Parameter'>
140600999828048
<class 'torch.nn.parameter.Parameter'>
1406009998

In [10]:
print(optimizer.param_groups[0]['params'][0] in optimizer.state)

True


In [15]:
for key in chkpt['optimizer'].state:
    if key.shape == model.layer1[0].conv1.weight.shape:
        print(key == model.layer1[0].conv1.weight)

tensor([[[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]]],


        ...,


        [[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],

In [17]:
len(chkpt['model'].keys())

101

In [19]:
for key in chkpt['model'].keys():
    print(key)

module.scale
module.layer1.0.num_batches_tracked
module.layer1.0.conv1.weight
module.layer1.0.bn1.weight
module.layer1.0.bn1.bias
module.layer1.0.bn1.running_mean
module.layer1.0.bn1.running_var
module.layer1.0.bn1.num_batches_tracked
module.layer1.0.conv2.weight
module.layer1.0.bn2.weight
module.layer1.0.bn2.bias
module.layer1.0.bn2.running_mean
module.layer1.0.bn2.running_var
module.layer1.0.bn2.num_batches_tracked
module.layer1.0.conv3.weight
module.layer1.0.bn3.weight
module.layer1.0.bn3.bias
module.layer1.0.bn3.running_mean
module.layer1.0.bn3.running_var
module.layer1.0.bn3.num_batches_tracked
module.layer1.0.downsample.0.weight
module.layer1.0.downsample.1.weight
module.layer1.0.downsample.1.bias
module.layer1.0.downsample.1.running_mean
module.layer1.0.downsample.1.running_var
module.layer1.0.downsample.1.num_batches_tracked
module.layer2.0.num_batches_tracked
module.layer2.0.conv1.weight
module.layer2.0.bn1.weight
module.layer2.0.bn1.bias
module.layer2.0.bn1.running_mean
modul

In [ ]:
model

In [21]:
chkpt['model']['module.layer2.0.num_batches_tracked']

tensor(542000, device='cuda:0')

In [13]:
type(list(chkpt['optimizer'].state.keys())[0])

torch.nn.parameter.Parameter

In [15]:
len(chkpt['optimizer'].state.keys())

49

In [23]:
for key in chkpt['optimizer'].state.keys():
    print(key.shape)

torch.Size([1])
torch.Size([64, 3, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 3, 1, 1])
torch.Size([64])
torch.Size([64])
torch.Size([160, 64, 3, 3])
torch.Size([160])
torch.Size([160])
torch.Size([160, 160, 3, 3])
torch.Size([160])
torch.Size([160])
torch.Size([160, 160, 3, 3])
torch.Size([160])
torch.Size([160])
torch.Size([160, 64, 1, 1])
torch.Size([160])
torch.Size([160])
torch.Size([320, 160, 3, 3])
torch.Size([320])
torch.Size([320])
torch.Size([320, 320, 3, 3])
torch.Size([320])
torch.Size([320])
torch.Size([320, 320, 3, 3])
torch.Size([320])
torch.Size([320])
torch.Size([320, 160, 1, 1])
torch.Size([320])
torch.Size([320])
torch.Size([640, 320, 3, 3])
torch.Size([640])
torch.Size([640])
torch.Size([640, 640, 3, 3])
torch.Size([640])
torch.Size([640])
torch.Size([640, 640, 3, 3])
torch.Size([640])
torch.Size([640])
torch.Size([640, 320, 1, 1])


In [24]:
chkpt['optimizer'].state[chkpt['model']['module.layer1.0.conv1.weight']]

{}

In [26]:
chkpt['model']['module.layer1.0.conv1.weight'].shape

torch.Size([64, 3, 3, 3])

In [5]:
for key in chkpt['optimizer'].state.keys():
    print(type(key))

<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.paramete

In [4]:
for key in chkpt['optimizer'].state.keys():
    try:
        if key.shape == torch.Size([64, 3, 3, 3]) and type(key) == torch.Tensor:
    #         print(key == chkpt['model']['module.layer1.0.conv1.weight'])
            print(chkpt['optimizer'].state[key])
    except:
        print(key)

In [32]:
chkpt['model']['module.layer1.0.conv1.weight']

tensor([[[[-1.7346e-03, -2.0156e-03, -2.3180e-03],
          [-1.7368e-03, -1.7394e-03, -1.7171e-03],
          [-1.8053e-03, -1.5829e-03, -1.2368e-03]],

         [[ 1.9253e-03,  6.6621e-04,  8.0381e-04],
          [ 1.3739e-03,  3.6057e-05,  6.8232e-04],
          [ 1.2967e-03,  6.4752e-04,  1.1699e-03]],

         [[ 4.7446e-03,  3.0351e-03,  3.2015e-03],
          [ 3.9372e-03,  2.0797e-03,  2.7595e-03],
          [ 3.6650e-03,  2.4927e-03,  2.9630e-03]]],


        [[[ 6.5395e-04,  6.2875e-03,  9.1429e-03],
          [-9.6274e-04,  3.6382e-02,  2.9371e-02],
          [ 1.6544e-02, -7.4395e-02, -1.6821e-02]],

         [[-4.7016e-03,  6.1646e-03,  7.6912e-03],
          [ 1.9596e-03,  5.7076e-02,  3.2797e-02],
          [ 3.5127e-02, -9.8689e-02, -2.8160e-02]],

         [[-7.3366e-03,  1.7887e-02,  1.6063e-02],
          [ 4.8400e-03,  3.1805e-02,  2.0513e-02],
          [ 1.3667e-02, -5.4773e-02, -3.2032e-02]]],


        [[[ 8.1392e-03, -2.6531e-03, -2.2012e-03],
          [ 6.0